# 🔗 Connexion VS Code ↔ Colab GPU Pro

**Objectif** : Connecter VS Code local à VM Colab (GPU L4/A100 avec Pro)

**Résultat** :
- ✅ Édition dans VS Code (confort, Copilot, breakpoints)
- ✅ Exécution sur GPU Colab **L4 (24GB)** ou **A100 (40GB)**
- ✅ Session 24h continue (vs 12h gratuit)
- ✅ Tous vos fichiers locaux accessibles
- ✅ Auto-détection GPU + optimisation batch size

---

## 🎯 GPU disponibles avec Colab Pro

| GPU | VRAM | Vitesse NSM | Usage recommandé |
|-----|------|-------------|------------------|
| T4 | 16 GB | 5 min | Debug/test rapide |
| **L4** | **24 GB** | **3 min** | ⭐ **Optimal (rapport qualité/prix)** |
| **A100** | **40 GB** | **1.5 min** | 🚀 **Production/benchmarks** |

**TPU v5e/v6e** : Déconseillé pour PyTorch (SentenceBERT incompatible)

---

## 📋 Prérequis

**Sur VS Code local** (vous avez déjà) :
- ✅ VS Code installé
- ✅ Extension "Remote - SSH" installée

**Sur Colab** (à exécuter ci-dessous) :
- Installation `colab-ssh`
- Création tunnel Cloudflare

## 🚀 Étape 1 : Installation et tunnel SSH

In [ ]:
# Installation colab-ssh
!pip install -q colab-ssh

print("✅ colab-ssh installé")

In [ ]:
# Lancer tunnel SSH avec Cloudflare
from colab_ssh import launch_ssh_cloudflared
import getpass

# Définir mot de passe pour connexion
password = "panini2025"  # ⚠️ Changez ce mot de passe !

print("🔧 Lancement tunnel SSH...")
print("⏳ Patientez 30 secondes...\n")

# Lancer SSH
launch_ssh_cloudflared(password=password)

print("\n" + "="*70)
print("✅ TUNNEL SSH ACTIF")
print("="*70)
print("\n📋 INSTRUCTIONS POUR VS CODE :")
print("\n1. Copiez la commande SSH ci-dessus (commence par 'ssh root@...')")
print("2. Dans VS Code :")
print("   - Ouvrir Command Palette (Ctrl+Shift+P)")
print("   - Taper 'Remote-SSH: Connect to Host...'")
print("   - Coller la commande SSH")
print("   - Entrer mot de passe : panini2025")
print("\n3. Une fois connecté :")
print("   - Ouvrir Terminal VS Code → vous êtes sur VM Colab !")
print("   - GPU T4 disponible : nvidia-smi")
print("   - Cloner repo : git clone https://github.com/stephanedenis/Panini-Research.git")
print("\n⚠️ IMPORTANT : Gardez cette fenêtre Colab ouverte !")
print("   Si vous fermez Colab → connexion VS Code perdue")
print("="*70)

## 🔍 Vérification GPU disponible

In [ ]:
# Vérifier GPU et recommandations
!nvidia-smi

import torch

print("\n" + "="*70)
print("🔍 DÉTECTION GPU ET OPTIMISATION")
print("="*70)

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    vram_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
    
    print(f"\n✅ PyTorch détecte GPU : {gpu_name}")
    print(f"   VRAM : {vram_gb:.1f} GB")
    
    # Recommandations selon GPU
    if "A100" in gpu_name:
        batch_size = 256
        temps_estime = "1.5 min"
        recommandation = "🚀 PRODUCTION - Optimal pour benchmarks multi-modèles"
    elif "L4" in gpu_name:
        batch_size = 128
        temps_estime = "3 min"
        recommandation = "⭐ OPTIMAL - Meilleur rapport qualité/prix"
    elif "T4" in gpu_name:
        batch_size = 64
        temps_estime = "5 min"
        recommandation = "✅ DEBUG - Rapide pour tests"
    elif "V100" in gpu_name:
        batch_size = 128
        temps_estime = "3.5 min"
        recommandation = "✅ PRODUCTION - Alternative A100"
    elif "P100" in gpu_name:
        batch_size = 96
        temps_estime = "4 min"
        recommandation = "⚠️ LEGACY - Upgrade vers L4 recommandé"
    else:
        batch_size = 32
        temps_estime = "8 min"
        recommandation = "⚠️ GPU inconnu - Paramètres conservateurs"
    
    print(f"\n📊 PARAMÈTRES OPTIMAUX pour NSM (60 primitives) :")
    print(f"   Batch size : {batch_size}")
    print(f"   Temps estimé : {temps_estime}")
    print(f"   Recommandation : {recommandation}")
    
    # Export variable pour usage ultérieur
    import os
    os.environ['NSM_BATCH_SIZE'] = str(batch_size)
    print(f"\n✅ Variable NSM_BATCH_SIZE={batch_size} exportée")
    
else:
    print("\n❌ Aucun GPU détecté")
    print("   💡 Solution : Runtime → Change runtime type → GPU")
    print("   💡 Pour Pro : Choisir L4 ou A100 dans les options")

print("="*70)

## 📦 Installation environnement NSM

In [ ]:
# Installer packages nécessaires
!pip install -q sentence-transformers scikit-learn matplotlib seaborn plotly pandas tqdm scipy

print("✅ Environnement Python prêt")
print("\n💡 Maintenant dans VS Code :")
print("   1. Ouvrir Terminal")
print("   2. git clone https://github.com/stephanedenis/Panini-Research.git")
print("   3. cd Panini-Research/semantic-primitives/notebooks")
print("   4. Ouvrir NSM_SentenceBERT_Local.ipynb")
print("   5. Exécuter cellules → GPU T4 utilisé !")

## ⏰ Maintenir session active

Cette cellule empêche Colab de déconnecter après 90 min d'inactivité.

In [ ]:
# Keep-alive automatique
import time
from IPython.display import clear_output

print("🔄 Keep-alive actif")
print("💡 Cette cellule maintient la session Colab active")
print("⚠️ NE PAS ARRÊTER cette cellule tant que vous travaillez dans VS Code\n")

try:
    while True:
        time.sleep(300)  # Ping toutes les 5 min
        clear_output(wait=True)
        current_time = time.strftime("%H:%M:%S")
        print(f"✅ Session active - {current_time}")
        print("💻 VS Code connecté via SSH")
        print("🔗 Tunnel : ACTIF")
        print("⏱️ Prochain ping dans 5 minutes...")
except KeyboardInterrupt:
    print("\n⚠️ Keep-alive arrêté")
    print("   Session Colab peut se déconnecter")

---

## 🎯 Workflow complet

### Dans Colab (cette fenêtre) :
1. ✅ Exécuter cellule 2 → tunnel SSH créé
2. ✅ Exécuter cellule 6 → keep-alive actif
3. ⚠️ **Laisser fenêtre ouverte** (minimiser OK, fermer = déconnexion)

### Dans VS Code (votre machine) :
1. `Ctrl+Shift+P` → "Remote-SSH: Connect to Host"
2. Coller commande SSH affichée dans cellule 2
3. Mot de passe : `panini2025`
4. Nouvelle fenêtre VS Code s'ouvre → **vous êtes sur VM Colab !**
5. Terminal VS Code : `git clone https://github.com/stephanedenis/Panini-Research.git`
6. Ouvrir `NSM_SentenceBERT_Local.ipynb`
7. Exécuter cellules → GPU T4 utilisé automatiquement

### Avantages :
- ✅ **Édition** : VS Code (Copilot, breakpoints, Git)
- ✅ **Exécution** : GPU T4 Colab (gratuit, rapide)
- ✅ **Debug** : Breakpoints fonctionnent sur code GPU
- ✅ **Fichiers** : Accès direct aux fichiers Colab
- ✅ **Terminal** : Comme si vous étiez en local

### Limitations :
- ⚠️ Session 12h max (puis reconnexion)
- ⚠️ Latence réseau (100-300ms)
- ⚠️ Fenêtre Colab doit rester ouverte

---

## 🐛 Troubleshooting

**Problème** : "Connection timed out"
- Solution : Réexécuter cellule 2 (tunnel SSH)

**Problème** : "Permission denied"
- Solution : Vérifier mot de passe (panini2025)

**Problème** : "Tunnel closed"
- Solution : Colab déconnecté → réexécuter notebook depuis début

**Problème** : "GPU not available" dans VS Code
- Solution : Vérifier Runtime → Change runtime type → GPU
- Pro : Choisir L4 ou A100 dans menu déroulant

**Problème** : "Out of memory" sur T4
- Solution : Réduire batch_size (32 au lieu de 64)
- Ou : Upgrade vers L4 (24GB) ou A100 (40GB)

---

## 📊 Benchmark performances (60 primitives NSM)

| Environnement | Temps encodage | Batch size | Coût |
|---------------|----------------|------------|------|
| **Local CPU** | 30 sec | 16 | $0 |
| **Colab T4** (gratuit) | 5 sec | 64 | $0 |
| **Colab L4** (Pro) | 3 sec | 128 | $10/mois |
| **Colab A100** (Pro) | 1.5 sec | 256 | $10/mois |

**Latence SSH** : +0.5 sec (négligeable)

**Recommandation** :
- Debug/test : **T4 gratuit** (suffisant)
- Production/benchmarks : **L4 Pro** (optimal)
- Multi-modèles : **A100 Pro** (si comparaison 5+ modèles simultanés)

**Évitez TPU** : PyTorch/SentenceBERT incompatible (TensorFlow only)